# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

1.- Examine the webpages and choose one that you want to work with.

Category choosen: https://www.zalando.fr/promo-sport-femme/


2.- Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

API endpoint: https://www.zalando.fr/api/catalog/articles?categories=promo-sport-femme&limit=84&offset=84&sort=sale

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

1.- Import libraries.

In [3]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

2.- Define the initial API endpoint URL.

In [9]:
# your code here
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
initial_endpoint = 'https://www.zalando.fr/api/catalog/articles?categories=promo-sport-femme&limit=84&offset=0&sort=sale'

3.- Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

In [10]:
flattened_data = pd.DataFrame()
response = requests.get(initial_endpoint,headers=header)
results = response.json()
results

{'total_count': 6099,
 'pagination': {'page_count': 73, 'current_page': 1, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'ME141B017-C11',
   'name': 'DISTRICT MENDI BACKSTRAP - Sandales de randonnée - turbulence',
   'price': {'original': '69,95\xa0€',
    'promotional': '49,00\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['36', '38', '39', '40', '41', '42'],
   'url_key': 'merrell-district-mendi-backstrap-sandales-de-randonnee-turbulence-me141b017-c11',
   'media': [{'path': 'ME/14/1B/01/7C/11/ME141B017-C11@2.jpg',
     'role': 'DEFAULT',
     'packet_shot': True}],
   'brand_name': 'Merrell',
   'is_premium': False,
   'family_articles': [{'sku': 'ME141B017-C11',
     'url_key': 'merrell-district-mendi-backstrap-sandales-de-randonnee-turbulence-me141b017-c11',
     'media': [{'path': 'ME/14/1B/01/7C/11/ME141B017-C11@2.jpg',
 

4.- Find out the total page count in the 1st page data

In [11]:
pages = results['pagination']['page_count']
pages

73

5.- Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

In [100]:
data = pd.DataFrame()
for i in range(pages):
    k = i*84
    url = f'https://www.zalando.fr/api/catalog/articles?categories=promo-sport-femme&limit=84&offset={k}&sort=sale'
    response = requests.get(url,headers=header)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data2 = json_normalize(flattened_data.articles[0]) #take the articles
    articles = flattened_data2.set_index('sku') #set sku as index
    data = data.append(articles,sort=False) #append the data of the page

6.- Print and review the data you obtained.

In [101]:
data.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key,outfits
sku,,,,,,,,,,,,,,,,,,,,
ME141B017-C11,NaN,Merrell,"[{'sku': 'ME141B017-C11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1B/01/7C/11/ME141B017-C11@2.j...,DISTRICT MENDI BACKSTRAP - Sandales de randonn...,False,False,False,False,"69,95 €","49,00 €",shoe,"[36, 38, 39, 40, 41, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-district-mendi-backstrap-sandales-de-r...,NaN
ME141A08H-Q11,NaN,Merrell,"[{'sku': 'ME141A08H-Q11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1A/08/HQ/11/ME141A08H-Q11@4.j...,YOKOTA 2 SPORT MID GTX - Chaussures de marche ...,False,False,False,False,"129,95 €","103,95 €",shoe,"[36, 37, 38.5, 39, 40, 41, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-yokota-2-sport-mid-gtx-chaussures-de-m...,NaN
ME141A08D-K11,NaN,Merrell,"[{'sku': 'ME141A08D-K11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1A/08/DK/11/ME141A08D-K11@10....,SIREN EDGE Q2 - Chaussures de marche - paloma/...,False,False,False,False,"94,95 €","75,95 €",shoe,"[36, 38, 38.5, 39, 40, 40.5, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-siren-edge-q2-chaussures-de-marche-pal...,NaN
N1241A0NO-Q11,NaN,Nike Performance,"[{'sku': 'N1241A0NO-Q11', 'url_key': 'nike-per...","[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/1A/0N/OQ/11/N1241A0NO-Q11@9.1...,REVOLUTION 4 - Chaussures de running neutres -...,False,True,True,False,"49,95 €","32,47 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40, 40.5,...",NaN,NaN,NaN,nike-performance-revolution-4-eu-chaussures-de...,NaN
EL941D01H-K11,NaN,Ellesse,"[{'sku': 'EL941D01H-K11', 'url_key': 'ellesse-...","[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'EL/94/1D/01/HK/11/EL941D01H-K11@8.j...,MORTER - T-shirt à manches longues - navy,False,False,False,False,"39,95 €","20,00 €",clothing,"[36, 38, 40, 42]",NaN,NaN,NaN,ellesse-morter-t-shirt-a-manches-longues-navy-...,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

#### Trending Brand

In [102]:
brands = pd.DataFrame(data['brand_name'].value_counts())

In [103]:
brands.head()

,brand_name
Nike Performance,686
adidas Performance,472
Puma,319
Reebok,282
Under Armour,188


#### Products with the highest discount

In [104]:
data[['price.original_num', 'price.promotional_num']] = data[['price.original','price.promotional']].replace('[\€,]','', regex=True).astype(int).div(100)

In [105]:
data.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,...,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key,outfits,price.original_num,price.promotional_num
sku,,,,,,,,,,,,,,,,,,,,,
ME141B017-C11,NaN,Merrell,"[{'sku': 'ME141B017-C11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1B/01/7C/11/ME141B017-C11@2.j...,DISTRICT MENDI BACKSTRAP - Sandales de randonn...,False,False,False,...,"49,00 €",shoe,"[36, 38, 39, 40, 41, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-district-mendi-backstrap-sandales-de-r...,NaN,69.95,49.00
ME141A08H-Q11,NaN,Merrell,"[{'sku': 'ME141A08H-Q11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1A/08/HQ/11/ME141A08H-Q11@4.j...,YOKOTA 2 SPORT MID GTX - Chaussures de marche ...,False,False,False,...,"103,95 €",shoe,"[36, 37, 38.5, 39, 40, 41, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-yokota-2-sport-mid-gtx-chaussures-de-m...,NaN,129.95,103.95
ME141A08D-K11,NaN,Merrell,"[{'sku': 'ME141A08D-K11', 'url_key': 'merrell-...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'ME/14/1A/08/DK/11/ME141A08D-K11@10....,SIREN EDGE Q2 - Chaussures de marche - paloma/...,False,False,False,...,"75,95 €",shoe,"[36, 38, 38.5, 39, 40, 40.5, 42]",https://ccp-et.metrigo.zalan.do/event/sbv?z=64...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,merrell-siren-edge-q2-chaussures-de-marche-pal...,NaN,94.95,75.95
N1241A0NO-Q11,NaN,Nike Performance,"[{'sku': 'N1241A0NO-Q11', 'url_key': 'nike-per...","[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/1A/0N/OQ/11/N1241A0NO-Q11@9.1...,REVOLUTION 4 - Chaussures de running neutres -...,False,True,True,...,"32,47 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40, 40.5,...",NaN,NaN,NaN,nike-performance-revolution-4-eu-chaussures-de...,NaN,49.95,32.47
EL941D01H-K11,NaN,Ellesse,"[{'sku': 'EL941D01H-K11', 'url_key': 'ellesse-...","[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'EL/94/1D/01/HK/11/EL941D01H-K11@8.j...,MORTER - T-shirt à manches longues - navy,False,False,False,...,"20,00 €",clothing,"[36, 38, 40, 42]",NaN,NaN,NaN,ellesse-morter-t-shirt-a-manches-longues-navy-...,NaN,39.95,20.00


In [106]:
data['discount'] =  data['price.original_num'] - data['price.promotional_num']

In [107]:
data.sort_values(by=['discount'])
data[['name','discount']].head()

,name,discount
sku,,
ME141B017-C11,DISTRICT MENDI BACKSTRAP - Sandales de randonn...,20.95
ME141A08H-Q11,YOKOTA 2 SPORT MID GTX - Chaussures de marche ...,26.00
ME141A08D-K11,SIREN EDGE Q2 - Chaussures de marche - paloma/...,19.00
N1241A0NO-Q11,REVOLUTION 4 - Chaussures de running neutres -...,17.48
EL941D01H-K11,MORTER - T-shirt à manches longues - navy,19.95


#### The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [108]:
sum_of_discounts = data['discount'].sum() / data['price.original_num'].sum()

In [109]:
print(f'Sum of discounts: {round(sum_of_discounts*100,2)}%')

Sum of discounts: 30.51%
